In [15]:
from faker import Faker
import json
import sqlite3
import random
import secrets

In [16]:
# get the salesman from the database
def get_salesman():
    conn = sqlite3.connect("database/database.db")
    cursor = conn.cursor()
    cursor.execute("SELECT id, name FROM employees WHERE role = 'Salesman'")
    salesman = cursor.fetchall()
    conn.close()
    return salesman

# customer crypto id
def customer_crypto_id():
    return secrets.token_hex(36)

# customer company name
def customer_company_name():
    fake = Faker(['en_US'])
    return fake.company()

# customer address
def customer_address():
    fake = Faker(['en_US', 'en_GB', 'en_AU'])
    return fake.address()

# get a random salesman id
def get_random_salesman_id(salesman):
    return random.choice(salesman)[0]

# using the random salesmand id get the name
def get_salesman_name(salesman, salesman_id):
    for s in salesman:
        if s[0] == salesman_id:
            return s[1]

# get cutomer email
def customer_email():
    fake = Faker()
    return fake.email()

# get customer phone number
def customer_phone_number():
    fake = Faker()
    return fake.phone_number()

# date of registration
def date_of_registration():
    fake = Faker()
    return fake.date_this_decade().strftime("%Y-%m-%d")

# fake first name
def fake_first_name():
    fake = Faker()
    return fake.first_name()

# fake last name
def fake_last_name():
    fake = Faker()
    return fake.last_name()

# fake city
def fake_city():
    fake = Faker()
    return fake.city()

# fake state
def fake_state():
    fake = Faker()
    return fake.state()

# fake zip code
def fake_zip_code():
    fake = Faker()
    return fake.zipcode()

# gacha customer type
def customer_type():
    return random.choice(["Business", "Personal"])

# fake tax number
def fake_tax_number():
    fake = Faker()
    return fake.random_int(min=100000000, max=999999999)

# fake identity number
def fake_identity_number():
    fake = Faker()
    return fake.random_int(min=100000000, max=999999999)

In [17]:
def create_customer():
    salesman = get_salesman()
    salesman_id = get_random_salesman_id(salesman)

    type = customer_type()
    
    if type == "Business":
        first_name = None
        last_name = None
        company_name = customer_company_name()
        ic_number = None
    else:
        first_name = fake_first_name()
        last_name = fake_last_name()
        company_name = None
        ic_number = fake_identity_number()
    
    customers = {
        'id': customer_crypto_id(),
        'first_name': first_name,
        'last_name': last_name,
        'company_name': company_name,
        'customer_type': type,
        'address': customer_address(),
        'city': fake_city(),
        'state': fake_state(),
        'zip_code': fake_zip_code(),
        'email': customer_email(),
        'phone_number': customer_phone_number(),
        'date_of_registration': date_of_registration(),
        'tax_number': fake_tax_number(),
        'identity_number': ic_number,
        'salesman_id': salesman_id,
        'salesman': get_salesman_name(salesman, salesman_id)
    }
    
    return customers

def multiple_customers(n):
    customers = [create_customer() for _ in range(n)]
    return json.dumps(customers, indent=4)


In [18]:
# testing
if __name__ == "__main__":
    print(multiple_customers(1))

[
    {
        "id": "9ecc2a7050de10f4ac32f85c21876beb9382c47b8f4991d7ef69279bdaaadfea73195a84",
        "first_name": null,
        "last_name": null,
        "company_name": "Mcguire PLC",
        "customer_type": "Business",
        "address": "927 Carroll Canyon Suite 155\nLaneport, ND 75179",
        "city": "North Michael",
        "state": "Alabama",
        "zip_code": "20639",
        "email": "oanderson@example.net",
        "phone_number": "001-996-228-0931x528",
        "date_of_registration": "2021-03-23",
        "tax_number": 731898167,
        "identity_number": null,
        "salesman_id": "82319ba0439ab2d6407cd5e25991203777a73e60e999cf512f12aa784bce8ae9bc605aca",
        "salesman": "James Anderson"
    }
]


In [19]:
# insert the customer into the database
conn = sqlite3.connect("database/database.db")
c = conn.cursor()

c.execute(
    '''
    CREATE TABLE IF NOT EXISTS customers (
        id TEXT PRIMARY KEY NOT NULL,
        first_name TEXT,
        last_name TEXT,
        company_name TEXT,
        customer_type TEXT NOT NULL,
        address TEXT NOT NULL,
        city TEXT NOT NULL,
        state TEXT NOT NULL,
        zip_code TEXT NOT NULL,
        email TEXT NOT NULL,
        phone_number TEXT NOT NULL,
        date_of_registration DATE NOT NULL,
        tax_number TEXT NOT NULL,
        identity_number TEXT,
        salesman_id TEXT NOT NULL,
        salesman TEXT NOT NULL,
        FOREIGN KEY (salesman_id) REFERENCES employees (id)
    )
    '''
)

json_data = multiple_customers(300)

# save the json data as json
with open('json_data/customers.json', 'w') as f:
    f.write(json_data)

for customer in json.loads(json_data):
    c.execute(
        '''
        INSERT INTO customers (id, first_name, last_name, company_name, customer_type, address, city, state, zip_code, email, phone_number, date_of_registration, tax_number, identity_number, salesman_id, salesman)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''',
        (
            customer['id'],
            customer['first_name'],
            customer['last_name'],
            customer['company_name'],
            customer['customer_type'],
            customer['address'],
            customer['city'],
            customer['state'],
            customer['zip_code'],
            customer['email'],
            customer['phone_number'],
            customer['date_of_registration'],
            customer['tax_number'],
            customer['identity_number'],
            customer['salesman_id'],
            customer['salesman']
        )
    )
    
conn.commit()
conn.close()


In [20]:
#conn = sqlite3.connect('database/database.db')
#c = conn.cursor()
#c.execute('DROP TABLE customers')
#conn.commit()